In [ ]:
!pip install -r requirements.txt

In [ ]:
import os, random, zipfile
import numpy as np
from PIL import Image
from pathlib import Path
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

import torch
import torch.nn.functional as F
from torch.cuda.amp import autocast
import torchvision.transforms as T
import torchvision.transforms.functional as TF
from transformers import SegformerForSemanticSegmentation

print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"GPU: {torch.cuda.get_device_name(0)}")

In [2]:
# Seed для воспроизводимости
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    os.environ['PYTHONHASHSEED'] = str(seed)

set_seed(42)
print("Seed: 42")

Seed: 42


In [3]:
# Конфигурация
class Config:
    BASE_DIR = Path(r'D:\Downloads\Hach_rosn')
    TRAIN_TARGET_DIR = BASE_DIR / 'data_mkGYPLg' / 'target'
    PREDICT_INPUT_DIR = BASE_DIR / 'predict_input'
    OUTPUT_DIR = BASE_DIR / 'output'
    MODELS_DIR = OUTPUT_DIR / 'models'
    PREDICTIONS_DIR = OUTPUT_DIR / 'predictions'
    
    MODEL_NAME = 'nvidia/mit-b5'
    NUM_CLASSES = 40
    TILE_SIZE = 640
    TILE_OVERLAP = 256
    DEVICE = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

config = Config()
config.PREDICTIONS_DIR.mkdir(exist_ok=True, parents=True)

print(f"Device: {config.DEVICE}")
print(f"Classes: {config.NUM_CLASSES}")
print(f"Predict input: {config.PREDICT_INPUT_DIR}")
print(f"Model path: {config.MODELS_DIR / 'best.pth'}")

Device: cuda
Classes: 40
Predict input: D:\Downloads\Hach_rosn\predict_input
Model path: D:\Downloads\Hach_rosn\output\models\best.pth


In [4]:
# Создание маппинга значений
mask_files = list(config.TRAIN_TARGET_DIR.glob('*.png'))[:500]
unique_vals = set()

for mf in tqdm(mask_files, desc='Scanning'):
    mask = np.array(Image.open(mf))
    unique_vals.update(np.unique(mask).tolist())

unique_vals = sorted(unique_vals)
print(f"\nУникальные значения: {unique_vals}")
print(f"Количество классов: {len(unique_vals)}")

# Маппинг: значение → класс
value_to_class = {val: idx for idx, val in enumerate(unique_vals)}
class_to_value = {idx: val for val, idx in value_to_class.items()}

print(f"\nМаппинг создан: {len(value_to_class)} значений → {len(unique_vals)} классов")
print(f"Примеры: {dict(list(value_to_class.items())[:5])}")

Scanning:   0%|          | 0/500 [00:00<?, ?it/s]


Уникальные значения: [0, 25, 30, 35, 40, 45, 50, 55, 60, 65, 70, 75, 80, 85, 90, 95, 100, 105, 110, 115, 120, 125, 130, 135, 140, 145, 150, 155, 160, 165, 170, 175, 180, 185, 190, 195, 200, 205, 210, 215]
Количество классов: 40

Маппинг создан: 40 значений → 40 классов
Примеры: {0: 0, 25: 1, 30: 2, 35: 3, 40: 4}


In [5]:
# Загрузка модели
print("Загрузка SegFormer")
model = SegformerForSemanticSegmentation.from_pretrained(
    config.MODEL_NAME, 
    num_labels=config.NUM_CLASSES, 
    ignore_mismatched_sizes=True
)
model = model.to(config.DEVICE)
print(f"✓ Модель загружена: {sum(p.numel() for p in model.parameters()):,} параметров")

Загрузка SegFormer


Some weights of SegformerForSemanticSegmentation were not initialized from the model checkpoint at nvidia/mit-b5 and are newly initialized: ['decode_head.batch_norm.bias', 'decode_head.batch_norm.num_batches_tracked', 'decode_head.batch_norm.running_mean', 'decode_head.batch_norm.running_var', 'decode_head.batch_norm.weight', 'decode_head.classifier.bias', 'decode_head.classifier.weight', 'decode_head.linear_c.0.proj.bias', 'decode_head.linear_c.0.proj.weight', 'decode_head.linear_c.1.proj.bias', 'decode_head.linear_c.1.proj.weight', 'decode_head.linear_c.2.proj.bias', 'decode_head.linear_c.2.proj.weight', 'decode_head.linear_c.3.proj.bias', 'decode_head.linear_c.3.proj.weight', 'decode_head.linear_fuse.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Модель загружена: 84,624,104 параметров


In [6]:
# Загрузка весов лучшей модели
best_model_path = config.MODELS_DIR / 'last.pth'

if not best_model_path.exists():
    raise FileNotFoundError(f"Файл модели не найден: {best_model_path}")

print(f"Загрузка весов из {best_model_path}...")
ckpt = torch.load(best_model_path, map_location=config.DEVICE, weights_only=False)
model.load_state_dict(ckpt['model_state_dict'])
model.eval()

print(f"\nМодель загружена:")
print(f"Epoch: {ckpt['epoch']+1}")
print(f"Val Dice: {ckpt['dice']:.4f}")
print(f"Best Dice: {ckpt.get('best_dice', ckpt['dice']):.4f}")

Загрузка весов из D:\Downloads\Hach_rosn\output\models\last.pth...

Модель загружена:
Epoch: 30
Val Dice: 0.9948
Best Dice: 0.9949


In [ ]:
# Predictor с ПРАВИЛЬНЫМ Gaussian Weighting (усреднение логитов, не классов!)
class Predictor:
    def __init__(self, model, device, class_to_value, tile_size=640, overlap=256):
        self.model = model
        self.device = device
        self.class_to_value = class_to_value
        self.num_classes = len(class_to_value)
        self.tile_size = tile_size
        self.overlap = overlap
        self.stride = tile_size - overlap
        self.norm = T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
    def predict_tile(self, tile_pil):
        """Возвращает ВЕРОЯТНОСТИ (softmax)"""
        tile = TF.to_tensor(tile_pil).unsqueeze(0).to(self.device)
        tile = self.norm(tile)
        with torch.no_grad(), autocast():
            logits = self.model(pixel_values=tile).logits
            logits = F.interpolate(logits, size=(self.tile_size, self.tile_size), 
                                   mode='bilinear', align_corners=False)
            # ВАЖНО: возвращаем вероятности, не классы!
            probs = F.softmax(logits, dim=1)
        return probs.cpu().numpy()[0]  # shape: (num_classes, H, W)
    
    def predict_large(self, img_pil):
        w, h = img_pil.size
        
        # Накопление вероятностей для каждого класса
        prob_sum = np.zeros((self.num_classes, h, w), dtype=np.float32)
        weight_sum = np.zeros((h, w), dtype=np.float32)
        
        # Создание Gaussian weight (плавное взвешивание)
        c = self.tile_size // 2
        y, x = np.ogrid[:self.tile_size, :self.tile_size]
        dist = np.sqrt((x-c)**2 + (y-c)**2)
        # Более плавное гауссовское ядро для лучшего смешивания
        sigma = self.tile_size / 4  # Можно настроить (2, 3, 4, 5)
        weight = np.exp(-dist**2 / (2*sigma**2))
        weight = weight / weight.max()
        
        # Sliding window
        n_h = (h - self.tile_size) // self.stride + 1
        n_w = (w - self.tile_size) // self.stride + 1
        
        for i in range(n_h):
            for j in range(n_w):
                y1, x1 = i*self.stride, j*self.stride
                y2, x2 = min(y1+self.tile_size, h), min(x1+self.tile_size, w)
                
                tile = img_pil.crop((x1, y1, x2, y2))
                if tile.size != (self.tile_size, self.tile_size):
                    padded = Image.new('RGB', (self.tile_size, self.tile_size))
                    padded.paste(tile, (0, 0))
                    tile = padded
                
                # Получаем вероятности (num_classes, tile_size, tile_size)
                tile_probs = self.predict_tile(tile)
                tile_probs = tile_probs[:, :y2-y1, :x2-x1]
                tile_w = weight[:y2-y1, :x2-x1]
                
                # Взвешенное накопление вероятностей
                prob_sum[:, y1:y2, x1:x2] += tile_probs * tile_w[np.newaxis, :, :]
                weight_sum[y1:y2, x1:x2] += tile_w
        
        # Усреднение вероятностей
        prob_avg = prob_sum / np.maximum(weight_sum[np.newaxis, :, :], 1e-6)
        
        # ТОЛЬКО ТЕПЕРЬ делаем argmax (на усредненных вероятностях!)
        pred_classes = prob_avg.argmax(axis=0).astype(np.uint8)
        
        # Обратный маппинг: класс → оригинальное значение
        pred_values = np.zeros_like(pred_classes)
        for cls, val in self.class_to_value.items():
            pred_values[pred_classes == cls] = val
        
        return pred_values

predictor = Predictor(model, config.DEVICE, class_to_value, config.TILE_SIZE, config.TILE_OVERLAP)
print("Predictor готов (с Gaussian Weighting на вероятностях)")

Predictor готов (с правильным Gaussian Weighting на вероятностях!)


In [8]:
# Поиск изображений
sub_imgs = sorted(list(config.PREDICT_INPUT_DIR.glob('*.png')))
print(f"\nНайдено изображений: {len(sub_imgs)}")
print(f"Путь: {config.PREDICT_INPUT_DIR}")

if len(sub_imgs) == 0:
    raise FileNotFoundError(f"Не найдены изображения в {config.PREDICT_INPUT_DIR}")

# Создать папку для масок
pred_dir = config.PREDICTIONS_DIR / 'predict_target'
if pred_dir.exists():
    import shutil
    shutil.rmtree(pred_dir)
pred_dir.mkdir(exist_ok=True)

print(f"Sliding Window: {config.TILE_SIZE}x{config.TILE_SIZE}, overlap={config.TILE_OVERLAP}")

# Генерация масок
for img_path in tqdm(sub_imgs, desc='Predictions'):
    img = Image.open(img_path).convert('RGB')
    pred_mask = predictor.predict_large(img)
    
    # Сохранить как grayscale uint8
    pred_mask_uint8 = pred_mask.astype(np.uint8)
    pred_image = Image.fromarray(pred_mask_uint8, mode='L')
    pred_image.save(pred_dir / img_path.name)

print(f"\nМаски сохранены: {pred_dir}")

# Создать архив
zip_path = config.OUTPUT_DIR / 'predict_target.zip'
if zip_path.exists():
    zip_path.unlink()

print(f"\nСоздание архива...")
with zipfile.ZipFile(zip_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
    for mask_file in sorted(pred_dir.glob('*.png')):
        zipf.write(mask_file, arcname=mask_file.name)

size_mb = zip_path.stat().st_size / (1024*1024)

print(f"Архив: {zip_path}")
print(f"Размер: {size_mb:.2f} MB")
print(f"Файлов: {len(list(pred_dir.glob('*.png')))}")

if size_mb > 6:
    print(f"ВНИМАНИЕ: Размер превышает 6 MB!")
else:
    print(f"Размер в пределах нормы (<6 MB)")

# Проверка архива
with zipfile.ZipFile(zip_path, 'r') as zipf:
    files_in_zip = zipf.namelist()
    print(f"\nПроверка архива:")
    print(f"   Файлов: {len(files_in_zip)}")
    print(f"   Первые 3: {files_in_zip[:3]}")
    print(f"   Последние 3: {files_in_zip[-3:]}")
    
    # Проверка формата
    if len(files_in_zip) > 0:
        with zipf.open(files_in_zip[0]) as f:
            test_img = Image.open(f)
            test_arr = np.array(test_img)
            print(f"\nПроверка формата ({files_in_zip[0]}):")
            print(f"   Размер: {test_img.size}")
            print(f"   Режим: {test_img.mode}")
            print(f"   Dtype: {test_arr.dtype}")
            print(f"   Значения: {test_arr.min()} - {test_arr.max()}")
            print(f"   Уникальных: {len(np.unique(test_arr))}")



Найдено изображений: 100
Путь: D:\Downloads\Hach_rosn\predict_input
Sliding Window: 640x640, overlap=256


Predictions:   0%|          | 0/100 [00:00<?, ?it/s]


Маски сохранены: D:\Downloads\Hach_rosn\output\predictions\predict_target

Создание архива...
Архив: D:\Downloads\Hach_rosn\output\predict_target.zip
Размер: 5.28 MB
Файлов: 100
Размер в пределах нормы (<6 MB)

Проверка архива:
   Файлов: 100
   Первые 3: ['0.png', '1.png', '10.png']
   Последние 3: ['97.png', '98.png', '99.png']

Проверка формата (0.png):
   Размер: (3000, 3000)
   Режим: L
   Dtype: uint8
   Значения: 0 - 210
   Уникальных: 20
